In [1]:
import sys
## sys.path.append('/home/ananth/nsf_data_ingestion/')
sys.path.append('/home/eileen/nsf_data_ingestion/')
from nsf_data_ingestion.config import nsf_config
from nsf_data_ingestion.objects import data_source_params
from nsf_data_ingestion.utils.utils_functions import get_last_load
import os
from shutil import copyfile
from shutil import rmtree
import tarfile
import shutil
import zipfile
from ftplib import FTP
import logging
import calendar
import time
from subprocess import call
logging.getLogger().setLevel(logging.INFO)

def download_med_data(param_list):
    medline_ftp_server = 'ftp.ncbi.nlm.nih.gov'
    #medline_ftp_server = param_list.get('ftp_server')
    medline_directory_path_data = '/home/eileen/airflow/medline_data_0/'
    #medline_directory_path_data = param_list.get('directory_path')
    #medline_medleasebaseline = '/nlmdata/.medleasebaseline/gz/'
    #medline_medleasebaseline = param_list.get('medleasebaseline_url')
    timestamp_file = 'time_stamp.txt'
    #timestamp_file = param_list.get('timestamp_file')
    medline_medlease_urls = ['/pubmed/baseline','/pubmed/updatefiles']
    #medline_medlease_urls = param_list.get('medlease_url')

#     medline_ftp_server = param_list.get('ftp_server')
#     medline_directory_path_data = param_list.get('directory_path')
#     ftp_path = param_list.get('medleasebaseline_url')
#     timestamp_file = param_list.get('timestamp_file')
#     medline_medlease_urls = param_list.get('medline_medlease_urls')
    
    last_load = get_last_load(medline_directory_path_data, timestamp_file)
    
    if 1:
        logging.info('Old Data. Downloading Updated Data')
        rmtree(medline_directory_path_data)
        os.makedirs(medline_directory_path_data)
        
        for medline_url in medline_medlease_urls:
            ftp = FTP(medline_ftp_server)
            ftp.login(user='', passwd='')
            ftp.cwd(medline_url)
            files = ftp.nlst()

            for file in files:
                if file.endswith('.xml.gz'):
                    localfile = open(medline_directory_path_data + file, 'wb')
#                     EOFError as e:
#                         continue:
                    ftp.retrbinary("RETR " + file, localfile.write)
                    logging.info('Downloading file - ' + file + ', from ' + medline_url + '. Pleae Wait.................')
            ftp.quit()
            localfile.close()
            
            logging.info('Updating TimeStamp')
            f = open(medline_directory_path_data + "time_stamp.txt", "a")
            cur_time = calendar.timegm(time.gmtime())
            f.write(str(cur_time))
            f.close()
        
def persist(param_list):
    medline_hdfs_path = '/user/eileen/medline/'
    #medline_hdfs_path = param_list.get('hdfs_path')
    medline_directory_path_data = '/home/eileen/airflow/medline_data_new/'
    #medline_directory_path_data = param_list.get('directory_path')
    logging.info('Persisting data to HDFS')
    if not call(["hdfs", "dfs", "-test", "-d", medline_hdfs_path]):
        call(["hdfs", "dfs", "-rm", "-r", "-f", medline_hdfs_path])

    call(["hdfs", "dfs", "-mkdir", medline_hdfs_path])
    call(["hdfs", "dfs", "-put", medline_directory_path_data, medline_hdfs_path])
    logging.info('Files Persisted to - %s', medline_hdfs_path)

def download(data_source_name):
    download_med_data(data_source_params.mapping.get(data_source_name))
                                                     
def persist_hdfs(data_source_name):
    persist(data_source_params.mapping.get(data_source_name))



In [31]:
download(nsf_config.medline)

INFO:root:Directory Path Exists
INFO:root:Timestamp Exists
INFO:root:Old Data. Downloading Updated Data
INFO:root:Downloading file - pubmed21n0002.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0012.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0022.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0032.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0042.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0052.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0062.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0072.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0082.xml.gz, from /pubmed/baseline. Pleae 

INFO:root:Downloading file - pubmed21n0802.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0812.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0822.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0832.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0842.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0852.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0862.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0872.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0882.xml.gz, from /pubmed/baseline. Pleae Wait.................
INFO:root:Downloading file - pubmed21n0892.xml.gz, from /pubmed/baseline. Pleae Wa

In [34]:
persist_hdfs(nsf_config.medline)

INFO:root:Persisting data to HDFS
INFO:root:Files Persisted to - /user/eileen/medline/


In [ ]:
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2/lib/spark2/')
import os, findspark

# temporarily changing PYSPARK_PYTHON to avoid rdd error.
os.environ['PYSPARK_PYTHON'] = '/home/tozeng/anaconda3/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/hzhuang/anaconda3/bin/python'
import sys
sys.path.append('/home/eileen/nsf_data_ingestion/')
import zipfile
import zipimport
import io
import logging
logging.getLogger().setLevel(logging.INFO)
from pyspark.sql import functions
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import Window
from pyspark.sql.functions import rank, max, sum, desc
import zlib
import importlib
import os
from os import path
from shutil import copyfile
from shutil import rmtree
from subprocess import call

#sys.path.append('/home/ananth/nsf_data_ingestion/')

from nsf_data_ingestion.config import spark_config
from nsf_data_ingestion.objects import data_source_params


def create_session(libraries_list):
    logging.info('Creating Spark Session....')
    spark = SparkSession.builder.config("spark.executor.instances", spark_config.exec_instance).\
                                 config("spark.executor.memory", spark_config.exec_mem).\
                                 config('spark.executor.cores', spark_config.exec_cores).\
                                 config('spark.cores.max', spark_config.exec_max_cores).\
                                 appName(data_source_name).getOrCreate()
    spark.sparkContext.addPyFile('/home/eileen/nsf_data_ingestion/libraries/pubmed_parser-0.1.0-py3.6.egg')
    spark.sparkContext.addPyFile('/home/eileen/nsf_data_ingestion/libraries/Unidecode-1.1.1-py3.6.egg')
#     for library in libraries_list:
#         logging.info('Adding Libraries' + str(library))
#         spark.sparkContext.addPyFile(library)    # adding libraries
#         spark.sparkContext.addPyFile(library)

        
    return spark

def parse_gzip_medline_str(gzip_str):
    import pubmed_parser as pp
    import unidecode
    print(pp.__file__)
    print(unidecode.__file__)
    
    filepath = gzip_str[0]
    gzip_content = gzip_str[1]
    _, file_name = path.split(filepath)
    # decompress gzip

    xml_string = gzip_content.split("\n", 3)[3];
    articles = pp.parse_medline_xml(xml_string)
    return [Row(file_name=file_name, **article_dict)
            for article_dict in articles]


if __name__ == '__main__':
    data_source_name = 'medline'
    params_list = data_source_params.mapping.get(data_source_name)
    #medline_xml_path = '/user/eileen/medline/medline_data/'
    medline_xml_path= params_list.get('xml_path')
    print(medline_xml_path)
    #medline_parquet_path = '/user/eileen/medline/parquet/'
    import pubmed_parser as pp
    medline_parquet_path= params_list.get('parquet_path')
    libraries_list = spark_config.libraries_list
    
    print("Reading from {} and writing to {}.".format(medline_xml_path, medline_parquet_path))
    
    spark = create_session(libraries_list)
    hdfs_data = spark.sparkContext.wholeTextFiles(os.path.join(medline_xml_path, '*.xml.gz'), minPartitions=10000)
    preprocess = hdfs_data.flatMap(parse_gzip_medline_str)
    medline_df = preprocess.toDF()
    
    window = Window.partitionBy(['pmid']).orderBy(desc('file_name'))
    ##only get the last version of documents
    last_medline_df = medline_df.select(
        max('delete').over(window).alias('is_deleted'),
        rank().over(window).alias('pos'), '*').\
        where('is_deleted = False and pos = 1').\
        drop('is_deleted').drop('pos').drop('delete')
    
    if not call(["hdfs", "dfs", "-test", "-d", medline_parquet_path]):
            logging.info('Parquet Files Exist Deleting .......')
            call(["hdfs", "dfs", "-rm", "-r", "-f", medline_parquet_path])
            
    logging.info('Writing New parquet Files .......')
    last_medline_df.write.parquet(medline_parquet_path)
    spark.stop()

INFO:root:Creating Spark Session....


/user/eileen/medline/medline_data/
Reading from /user/eileen/medline/medline_data/ and writing to /user/eileen/medline/parquet/.


In [3]:
import sys
## sys.path.append('/home/ananth/nsf_data_ingestion/')
sys.path.append('/home/eileen/nsf_data_ingestion/')
from nsf_data_ingestion.config import nsf_config
from nsf_data_ingestion.objects import data_source_params
from nsf_data_ingestion.utils.utils_functions import get_last_load
import os
from shutil import copyfile
from shutil import rmtree
import tarfile
import shutil
import zipfile
from ftplib import FTP
import logging
import calendar
import time
from subprocess import call
logging.getLogger().setLevel(logging.INFO)
def persist(param_list):
    medline_hdfs_path = '/user/eileen/test'
    #medline_hdfs_path = param_list.get('hdfs_path')
    medline_directory_path_data = '/home/eileen/airflow/test/'
    #medline_directory_path_data = param_list.get('directory_path')
    logging.info('Persisting data to HDFS')
    if not call(["hdfs", "dfs", "-test", "-d", medline_hdfs_path]):
        call(["hdfs", "dfs", "-rm", "-r", "-f", medline_hdfs_path])

    call(["hdfs", "dfs", "-mkdir", medline_hdfs_path])
    call(["hdfs", "dfs", "-put", medline_directory_path_data, medline_hdfs_path])
    logging.info('Files Persisted to - %s', medline_hdfs_path)

def download(data_source_name):
    download_med_data(data_source_params.mapping.get(data_source_name))
                                                     
def persist_hdfs(data_source_name):
    persist(data_source_params.mapping.get(data_source_name))

In [4]:

persist_hdfs(nsf_config.medline)

INFO:root:Persisting data to HDFS
INFO:root:Files Persisted to - /user/eileen/test


In [ ]:
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2-2.4.0.cloudera2-1.cdh5.13.3.p0.1041012/lib/spark2/')
import os, findspark
os.environ['PYSPARK_PYTHON'] = '/home/tozeng/anaconda3/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/tozeng/anaconda3/bin/python'
import sys
sys.path.append('/home/eileen/nsf_data_ingestion/')
import zipfile
import zipimport
import io
import logging
logging.getLogger().setLevel(logging.INFO)
from pyspark.sql import functions
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import Window
from pyspark.sql.functions import rank, max, sum, desc
import zlib
import importlib
import os
from os import path
from shutil import copyfile
from shutil import rmtree
from subprocess import call

#sys.path.append('/home/ananth/nsf_data_ingestion/')

from nsf_data_ingestion.config import spark_config
from nsf_data_ingestion.objects import data_source_params


def create_session(libraries_list):
    logging.info('Creating Spark Session....')
    spark = SparkSession.builder.config("spark.executor.instances", spark_config.exec_instance).\
                                 config("spark.executor.memory", spark_config.exec_mem).\
                                 config('spark.executor.cores', spark_config.exec_cores).\
                                 config('spark.cores.max', spark_config.exec_max_cores).\
                                 appName(data_source_name).getOrCreate()
    spark.sparkContext.addPyFile('/home/eileen/nsf_data_ingestion/libraries/pubmed_parser-0.1.0-py3.6.egg')
    spark.sparkContext.addPyFile('/home/eileen/nsf_data_ingestion/libraries/Unidecode-1.1.1-py3.6.egg')
#         logging.info('Adding Libraries' + str(library))
#         spark.sparkContext.addPyFile(library)    # adding libraries
#         spark.sparkContext.addPyFile(library)

        
    return spark

def parse_gzip_medline_str(gzip_str):
    import pubmed_parser as pp
    import unidecode
    print(pp.__file__)
    print(unidecode.__file__)
    
    filepath = gzip_str[0]
    gzip_content = gzip_str[1]
    _, file_name = path.split(filepath)
    # decompress gzip

    xml_string = gzip_content.split("\n", 3)[3];
    articles = pp.parse_medline_xml(xml_string)
    return [Row(file_name=file_name, **article_dict)
            for article_dict in articles]


if __name__ == '__main__':
    data_source_name = 'medline'
    params_list = data_source_params.mapping.get(data_source_name)
    medline_xml_path = '/user/eileen/test/test'
    #medline_xml_path= params_list.get('xml_path')
    print(medline_xml_path)
    #medline_parquet_path = '/user/eileen/medline/parquet/'
    import pubmed_parser as pp
    medline_parquet_path= params_list.get('parquet_path')
    libraries_list = spark_config.libraries_list
    
    print("Reading from {} and writing to {}.".format(medline_xml_path, medline_parquet_path))
    
    spark = create_session(libraries_list)
    hdfs_data = spark.sparkContext.wholeTextFiles(os.path.join(medline_xml_path, '*.xml.gz'), minPartitions=10000)
    preprocess = hdfs_data.flatMap(parse_gzip_medline_str)
    medline_df = preprocess.toDF()
    
    window = Window.partitionBy(['pmid']).orderBy(desc('file_name'))
    ##only get the last version of documents
    last_medline_df = medline_df.select(
        max('delete').over(window).alias('is_deleted'),
        rank().over(window).alias('pos'), '*').\
        where('is_deleted = False and pos = 1').\
        drop('is_deleted').drop('pos').drop('delete')
    
    if not call(["hdfs", "dfs", "-test", "-d", medline_parquet_path]):
            logging.info('Parquet Files Exist Deleting .......')
            call(["hdfs", "dfs", "-rm", "-r", "-f", medline_parquet_path])
            
    logging.info('Writing New parquet Files .......')
    last_medline_df.write.parquet(medline_parquet_path)
    spark.stop()

INFO:root:Creating Spark Session....


/user/eileen/test/test
Reading from /user/eileen/test/test and writing to /user/eileen/medline/parquet/.
